In [1]:
#from utils.molecule_tools import get_molecule
from utils.cs_vqe_tools_original import greedy_dfs
from cs_vqe.circuit import cs_vqe_circuit
import json

/home/tim/anaconda3/lib/python3.7/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/home/tim/anaconda3/lib/python3.7/site-packages/qiskit/aqua/components/optimizers/optimizer.py:50: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  'qiskit.algorithms.optimizers', 'qiskit-terra')


In [2]:
file = 'molecule_data'
with open('data/'+file+'.json', 'r') as json_file:
    molecule_data = json.load(json_file)

with open('data/anz_circ_depth.json', 'r') as json_file:
    anz_data = json.load(json_file)
    
speciesname_list = [mol for mol in anz_data if (anz_data[mol]['num_qubits']!=anz_data[mol]['chemaccnum'] and not anz_data[mol]['chemaccnum']==0) and mol.find('alt')==-1]
speciesname_list

['HeH+_6-311G_SINGLET',
 'H2O_STO-3G_SINGLET',
 'BeH+_STO-3G_SINGLET',
 'LiH_STO-3G_SINGLET',
 'CH+_STO-3G_SINGLET',
 'HF_STO-3G_SINGLET',
 'B+_STO-3G_SINGLET',
 'OH+_STO-3G_SINGLET',
 'CH2_STO-3G_SINGLET',
 'BeH2_STO-3G_SINGLET',
 'Be_STO-3G_SINGLET',
 'C_STO-3G_SINGLET',
 'NH_STO-3G_SINGLET',
 'BH_STO-3G_SINGLET',
 'NH2+_STO-3G_SINGLET',
 'BH2+_STO-3G_SINGLET']

In [3]:
with open('data/old_model_data.json', 'r') as json_file:
    old_model_data = json.load(json_file)

In [4]:
model_data = {}

for speciesname in speciesname_list:
    print(speciesname)

    molecule    = old_model_data[speciesname]#get_molecule(speciesname, taper=True)
    num_qubits  = molecule['num_qubits']
    ham         = molecule['ham']
    uccsd       = molecule['uccsd']
    hf_config   = molecule['hf_config']
    num_tapered = molecule['num_tapered']
    true_gs_nrg = molecule['true_gs_nrg']
    #true_gs_vec = molecule['true_gs_vec']

    #for p in ham.keys():
    #    ham[p] = ham[p].real    
    #for p in uccsd.keys():
    #    uccsd[p] = uccsd[p].imag

    terms_noncon = old_model_data[speciesname]['terms_noncon']#greedy_dfs(ham, 10, criterion='size')[-1]

    print('Number of qubits:', num_qubits)
    mol_circ = cs_vqe_circuit(hamiltonian = ham,
                              terms_noncon= terms_noncon,
                              num_qubits  = num_qubits,
                              hf_config   = hf_config)

    ham_rotations = mol_circ.ham_rotations
    noncon         = mol_circ.noncon
    truegs         = mol_circ.truegs
    G              = mol_circ.G
    A              = mol_circ.A
    X_index        = mol_circ.X_index
    X_qubit        = mol_circ.X_qubit
    cs_vqe_energy  = mol_circ.cs_vqe_energy
    cs_vqe_errors  = mol_circ.cs_vqe_errors
    chem_acc_num_q = mol_circ.chem_acc_num_q
    order          = mol_circ.order
    ham_reduced    = mol_circ.ham_reduced
    reference_state= mol_circ.reference_state()

    model_data[speciesname] = {"ham":ham,
                               "uccsd":uccsd,
                               "num_qubits":num_qubits,
                               "hf_config":hf_config,
                               "terms_noncon":terms_noncon,
                               "num_tapered":num_tapered,
                               "true_gs_nrg":true_gs_nrg,
                               #"true_gs_vec":true_gs_vec,
                               "ham_rotations":ham_rotations,
                               "noncon":noncon,
                               "truegs":truegs,
                               "G":G,
                               "A":A,
                               "X_index":X_index,
                               "X_qubit":X_qubit,
                               "cs_vqe_energy":cs_vqe_energy,
                               "cs_vqe_errors":cs_vqe_errors,
                               "chem_acc_num_q":chem_acc_num_q,
                               "order":order,
                               "ham_reduced":ham_reduced,
                               "reference_state":reference_state
                                }

    print('Molecule constructed.\n')


HeH+_6-311G_SINGLET
Number of qubits: 10
Molecule constructed.

H2O_STO-3G_SINGLET
Number of qubits: 10
Molecule constructed.

BeH+_STO-3G_SINGLET
Number of qubits: 8
Molecule constructed.

LiH_STO-3G_SINGLET
Number of qubits: 8
Molecule constructed.

CH+_STO-3G_SINGLET
Number of qubits: 8
Molecule constructed.

HF_STO-3G_SINGLET
Number of qubits: 8
Molecule constructed.

B+_STO-3G_SINGLET
Number of qubits: 5
Molecule constructed.

OH+_STO-3G_SINGLET
Number of qubits: 8
Molecule constructed.

CH2_STO-3G_SINGLET
Number of qubits: 10
Molecule constructed.

BeH2_STO-3G_SINGLET
Number of qubits: 9
Molecule constructed.

Be_STO-3G_SINGLET
Number of qubits: 5
Molecule constructed.

C_STO-3G_SINGLET
Number of qubits: 5
Molecule constructed.

NH_STO-3G_SINGLET
Number of qubits: 8
Molecule constructed.

BH_STO-3G_SINGLET
Number of qubits: 8
Molecule constructed.

NH2+_STO-3G_SINGLET
Number of qubits: 10
Molecule constructed.

BH2+_STO-3G_SINGLET
Number of qubits: 10
Molecule constructed.



In [5]:
#data={}
#for k in model_data.keys():
#    data[str(k)] = model_data[k]
     
with open("data/model_data.json", "w") as outfile: 
    json.dump(model_data, outfile)